<a href="https://colab.research.google.com/github/rajivverma11/AI-AgentsOverview/blob/main/Langchain_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install LangChain Hub, a repository for storing and sharing LangChain prompts and chains
!pip install langchainhub

# Install the LangChain OpenAI integration, which provides easy access to OpenAI models
!pip install langchain-openai

# Install the core LangChain library, which provides tools for building applications with LLMs
!pip install langchain

# Install FAISS (Facebook AI Similarity Search), a library for efficient similarity search and clustering of dense vectors
!pip install faiss-cpu

# Upgrade LangChain Community and install Tavily Python, which provides web search functionalities for LangChain
!pip install -U langchain-community tavily-python

# Install a specific version of Gradio Client (0.2.10), a package for interacting with Gradio interfaces programmatically
!pip install gradio_client==0.2.10

# Install Gradio version 3.38.0, a library for creating web-based UI components for AI models and applications
!pip install gradio==3.38.0

# Install LangChain Community
!pip install langchain_community


In [ ]:
# Import KaggleHub, a library for accessing datasets and models from Kaggle
import kagglehub

# Import CSV module for reading and writing CSV files
import csv

# Import pandas for data manipulation and analysis
import pandas as pd

# Import math module for mathematical operations
import math

# Import NumPy for numerical computations and working with arrays
import numpy as np

# Import os module for interacting with the operating system
import os

# Import getpass module for securely handling user input (e.g., passwords)
import getpass

# Import StrOutputParser from LangChain Core to process and parse output strings
from langchain_core.output_parsers import StrOutputParser

# Import Google Drive module for mounting Google Drive (Used in Google Colab)
from google.colab import drive  # Used for mounting Google Drive

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(first_int:int, second_int:int) -> int:
  """Multiplies two integers"""
  return first_int * second_int


In [ ]:
print(multiply.name)
print(multiply.description)

result1= multiply.invoke({"first_int":2, "second_int":3})
print(result1)

multiply
Multiplies two integers
6


In [ ]:
from langchain.tools.render import render_text_description

render_tools = render_text_description([multiply])
print(render_tools)

multiply(first_int: int, second_int: int) -> int - Multiplies two integers


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""you are assistant with following tools:
{render_tools}
Given a user query, return tool name and arguments in JSON format with 'name' and 'arguments' keys.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])




In [ ]:
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
model=ChatOpenAI(model='gpt-4o-mini', temperature=0)

chain=prompt|model
result = chain.invoke({"input":"what is 4 times 5"})
print(result.content)

OpenAI API Key:··········
{
  "name": "multiply",
  "arguments": {
    "first_int": 4,
    "second_int": 5
  }
}


In [ ]:
from langchain_core.output_parsers import JsonOutputParser

parsed_output_chain = prompt | model | JsonOutputParser()
parsed_output_result = parsed_output_chain.invoke({"input":"what is 13 times 5"})
print(parsed_output_result)


{'name': 'multiply', 'arguments': {'first_int': 13, 'second_int': 5}}


In [ ]:
from operator import itemgetter
chain = prompt | model | JsonOutputParser() | itemgetter("arguments") | multiply

final_result = chain.invoke({"input":"what is 13 times 5"})
print(final_result)

65


In [ ]:
from langchain_core.tools import tool

@tool
def add(first_int:int, second_int:int) -> int:
  """Adds two integers"""
  return first_int + second_int

@tool
def subtract(first_int:int, second_int:int) -> int:
  """Subtracts two integers"""
  return first_int - second_int

@tool
def divide(first_int:int, second_int:int) -> int:
  """Divides two integers"""
  return first_int / second_int

@tool
def exponential (first_int:int, second_int:int) -> int:
  """Exponentiates two integers"""
  return first_int ** second_int

@tool
def modulo(first_int: int, second_int: int) -> int:
    """Find the remainder when the first integer is divided by the second integer."""
    if second_int == 0:
        raise ValueError("Modulo by zero is not allowed.")
    return first_int % second_int

In [ ]:
tools = [multiply, add, subtract, divide, exponential,modulo]
render_tools = render_text_description(tools)
print(render_tools)

system_prompt=f"""You are an assistant with following tools
{render_tools}
Given a user query, return tool name and arguments in JSON format with 'name' and 'arguments' keys.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])


multiply(first_int: int, second_int: int) -> int - Multiplies two integers
add(first_int: int, second_int: int) -> int - Adds two integers
subtract(first_int: int, second_int: int) -> int - Subtracts two integers
divide(first_int: int, second_int: int) -> int - Divides two integers
exponential(first_int: int, second_int: int) -> int - Exponentiates two integers
modulo(first_int: int, second_int: int) -> int - Find the remainder when the first integer is divided by the second integer.


In [ ]:
# Define a function to dynamically select and execute the appropriate tool
def tool_chain(model_output):
  tool_map = {tool.name:tool for tool in tools}
  choosen_tool = tool_map[model_output["name"]]
  return itemgetter('arguments') | choosen_tool


In [ ]:
from langchain_core.runnables import RunnablePassthrough
chain = prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=tool_chain)

final_result = chain.invoke({"input":"what is 3 plus 312"})
print(final_result)

{'name': 'add', 'arguments': {'first_int': 3, 'second_int': 312}, 'output': 315}


In [ ]:
# Above version in elaborative way

# llm_input = prompt.invoke({"input": "what is 3 plus 312"})
# print(f"llm_input: {llm_input}")
# print('\n')

# model_response = model.invoke(llm_input)
# print(f"model_response: {model_response}")
# print('\n')

# parsed = JsonOutputParser().invoke(model_response)
# print(f"parsed: {parsed}")
# print('\n')

# final = parsed.copy()
# final["output"] = tool_chain(parsed)
# print(final)
# print('\n')

# final["output"] = tool_chain(parsed)
# print(final)


llm_input: messages=[SystemMessage(content="You are an assistant with following tools\nmultiply(first_int: int, second_int: int) -> int - Multiplies two integers\nadd(first_int: int, second_int: int) -> int - Adds two integers\nsubtract(first_int: int, second_int: int) -> int - Subtracts two integers\ndivide(first_int: int, second_int: int) -> int - Divides two integers\nexponential(first_int: int, second_int: int) -> int - Exponentiates two integers\nGiven a user query, return tool name and arguments in JSON format with 'name' and 'arguments' keys.\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is 3 plus 312', additional_kwargs={}, response_metadata={})]


model_response: content='{\n  "name": "add",\n  "arguments": {\n    "first_int": 3,\n    "second_int": 312\n  }\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 148, 'total_tokens': 181, 'completion_tokens_details': {'accepted_prediction

In [ ]:
# tests

# Test cases
print(chain.invoke({"input": "What is 15 divided by 3?"}))  # Expected output: 5.0
print('\n')
print(chain.invoke({"input": "What is 7 modulo 4?"}))        # Expected output: 3
print('\n')
print(chain.invoke({"input": "What is 20 minus 8?"}))        # Expected output: 12
print('\n')
print(chain.invoke({"input": "What is 2 to the power of 5?"})) # Expected output: 32

{'name': 'divide', 'arguments': {'first_int': 15, 'second_int': 3}, 'output': 5.0}


{'name': 'modulo', 'arguments': {'first_int': 7, 'second_int': 4}, 'output': 3}


{'name': 'subtract', 'arguments': {'first_int': 20, 'second_int': 8}, 'output': 12}


{'name': 'exponential', 'arguments': {'first_int': 2, 'second_int': 5}, 'output': 32}
